<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_08/LangGraph%20Agent%20Design%20Patterns/Pattern_3__Routing_(LangGraph)_with_LinkedIn_Content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Install dependencies

In [1]:
!pip install -q langgraph langchain-openai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 19.1 MB/s eta 0:00:00


In [5]:
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langsmith import traceable

## Step 2: Set Environment for LangSmith Tracing

In [2]:
import os
import getpass

#  Prompt user to enter keys securely (not visible in UI)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "Routing"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


##  Step 3: Initialize LLM

In [8]:
!pip install -q langchain-community
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


<ipython-input-8-1e9109a9b616>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [9]:
from typing import TypedDict, Literal

class RoutingState(TypedDict):
    request: str
    decision: Literal["tip", "announcement", "post"]
    output: str


## Step 4: Define the Router Node

In [10]:
def route_marketing_request(state: RoutingState) -> dict:
    prompt = ChatPromptTemplate.from_template(
        """Classify this LinkedIn content request as 'tip', 'announcement', or 'post':

Request: {request}

Respond with only one word."""
    )
    chain = prompt | llm
    result = chain.invoke({"request": state["request"]}).content.lower().strip()
    return {"decision": result}


## Step 6: Define Writer Nodes

In [11]:
def tip_writer(state: RoutingState):
    prompt = ChatPromptTemplate.from_template(
        "Write 3 bullet point tips for LinkedIn based on this topic: {request}"
    )
    return {"output": (prompt | llm).invoke({"request": state["request"]}).content}

def announcer(state: RoutingState):
    prompt = ChatPromptTemplate.from_template(
        "Write a short, professional LinkedIn announcement about this: {request}"
    )
    return {"output": (prompt | llm).invoke({"request": state["request"]}).content}

def post_writer(state: RoutingState):
    prompt = ChatPromptTemplate.from_template(
        "Write a compelling LinkedIn post about: {request}"
    )
    return {"output": (prompt | llm).invoke({"request": state["request"]}).content}


## Step 7: Build the LangGraph

In [12]:
builder = StateGraph(RoutingState)

# Add all nodes
builder.add_node("router", route_marketing_request)
builder.add_node("tip_node", tip_writer)
builder.add_node("announcement_node", announcer)
builder.add_node("post_node", post_writer)

# Entry point
builder.set_entry_point("router")

# Add conditional routing logic
builder.add_conditional_edges("router", lambda state: state["decision"], {
    "tip": "tip_node",
    "announcement": "announcement_node",
    "post": "post_node"
})

# All writers lead to END
builder.add_edge("tip_node", END)
builder.add_edge("announcement_node", END)
builder.add_edge("post_node", END)

# Compile workflow
workflow = builder.compile()


## Step 8: Try with a Real Example

In [15]:
from langchain.prompts import ChatPromptTemplate

In [16]:
inputs = {"request": "Share 3 practical tips for using ChatGPT to improve productivity at work"}

final_state = workflow.invoke(inputs)

from pprint import pprint
pprint(final_state)


{'decision': 'tip',
 'output': '- Use ChatGPT to quickly generate ideas or solutions for '
           'work-related tasks or projects, saving time and boosting '
           'productivity.\n'
           '- Utilize ChatGPT to streamline communication with colleagues or '
           'clients by generating clear and concise messages or responses.\n'
           '- Leverage ChatGPT to automate repetitive tasks or processes, '
           'freeing up time to focus on more strategic or high-priority work.',
 'request': 'Share 3 practical tips for using ChatGPT to improve productivity '
            'at work'}
